Se desea crear un sistema de compras que, dado un conjunto de productos (con stock limitado),
nos permita añadirlos a un carrito de la compra. Las funcionalidades son
las siguientes:
• Crear un carrito de compra
• Añadir productos al carrito
• Controlar el stock de los productos
• Eliminar productos de carrito (total o parcialmente)
• Mostrar información de carrito: productos, coste por producto, coste total, etc.



In [ ]:
#Para este ejercicio necesitaremos varias clases distintas.

#La primera sin lugar a dudas será producto con atributos el nombre la cantidad en stock y el precio

#La segunda la tienda donde se almacenan los productos cuyo acceso a modificaciones solo tendrá el dueño

#La tercera será el carrito del cliente que va seleccionando los productos

#Y la cuarta la unión de todas las anteriores, el sistema de compras.
#El cliente solo tendrá acceso a las funciones de esta cuarta clase


class Producto:
    def __init__(self, nombre, precio, stock):#Atributos de la clase
        self.nombre = nombre
        self.precio = precio
        self.stock = stock

    def __str__(self):#Para el print
        return f"{self.nombre} - Precio: {self.precio}, Stock: {self.stock}"

class Tienda: #Esta clase solo tendría acceso a ella el responsable del negocio
    def __init__(self):
        self.productos = {}  # Diccionario como base de datos para almacenar productos y cantidades

    def añadir_producto(self, producto, cantidad):
        if producto.nombre in self.productos: # Si ya existe, se aumenta su stock
            self.productos[producto.nombre].stock += cantidad
        else: # Si es un nuevo producto, se añade con la cantidad especificada en el propio atributo
            producto.stock = cantidad #Necesario actualizar el atributo de la instancia si es cero
            #Ya que quizás no se ha hecho manualmente desde fuera
            self.productos[producto.nombre] = producto
        print(f"Producto {producto.nombre} añadido o actualizado en la tienda con {cantidad} unidades.")

    def eliminar_producto(self, nombre_producto, eliminar_totalmente=False):#True solo para el dueño
        if nombre_producto in self.productos:
            if eliminar_totalmente: # Eliminar completamente el producto de la tienda por el dueño
                del self.productos[nombre_producto]
                print(f"Producto {nombre_producto} eliminado completamente de la tienda.")
            else: # Mantener el producto con stock 0
                self.productos[nombre_producto].stock = 0
                print(f"Producto {nombre_producto} ahora tiene stock 0 pero sigue en la tienda.")
        else:
            print(f"Producto {nombre_producto} no existe en la tienda.")

    def buscar_producto(self, nombre_producto):#Funcion para el sistema del cliente
        return self.productos.get(nombre_producto, None)

    def mostrar_tienda(self):
        print("Tienda actual:")
        for producto in self.productos.values():#Mostrar los productos de la tienda
            print(producto)

class Carrito:
    def __init__(self):
        self.productos = {}  # Diccionario con nombres de productos como clave

    def añadir_producto(self, producto, cantidad):
        if producto.nombre in self.productos:#Si el producto ya lo tenemos, incrementar cantidad
            self.productos[producto.nombre]["cantidad"] += cantidad
        else:#Sino añadir un producto nuevo
            self.productos[producto.nombre] = {"producto": producto, "cantidad": cantidad}

    def eliminar_producto(self, producto, cantidad=None):
        nombre_producto = producto.nombre
        if nombre_producto in self.productos:#Si tenemos ese producto, procedemos a eliminar
            if cantidad is None or self.productos[nombre_producto]["cantidad"] <= cantidad:
                del self.productos[nombre_producto] #Eliminamos el producto del carrito
            else: #Sino quitamos las cantidades deseadas
                self.productos[nombre_producto]["cantidad"] -= cantidad

    def mostrar_informacion(self):
        total = 0 #Para reemplazar en el bucle
        print("Carrito de compras:")
        for producto_info in self.productos.values():#Buscar en todo el diccionario del carrito
            producto = producto_info["producto"]
            cantidad = producto_info["cantidad"]
            coste_producto = producto.precio * cantidad
            total += coste_producto
            print(f"{producto.nombre}: {cantidad} unidades - Total: {coste_producto}€")#Print de todo
        print(f"Coste total: {total}€")


class SistemaDeCompras:#Clase que tiene acceso el cliente y opera con el resto de las clases definidas previamente
    def __init__(self, tienda):
        self.tienda = tienda  # Accede a la tienda de ese sistema

    def buscar_producto(self, nombre_producto):
        return self.tienda.buscar_producto(nombre_producto)

    def añadir_al_carrito(self, carrito, nombre_producto, cantidad):
        producto = self.buscar_producto(nombre_producto)#Busca el producto y si existe ejecuta accion
        if producto:
            if producto.stock >= cantidad:
                carrito.añadir_producto(producto, cantidad)
                producto.stock -= cantidad #Actualiza el stock de la tienda
                print(f"Producto '{nombre_producto}' añadido al carrito.")
            else:
                print(f"Stock insuficiente de '{nombre_producto}'.")
        else:
            print(f"Producto '{nombre_producto}' no encontrado.")

    def eliminar_del_carrito(self, carrito, nombre_producto, cantidad=None):
        producto = self.buscar_producto(nombre_producto)
        if producto and nombre_producto in carrito.productos:
            producto_info = carrito.productos[nombre_producto]
            cantidad_actual = producto_info["cantidad"]

            if cantidad is None or cantidad >= cantidad_actual:
                carrito.eliminar_producto(producto)  # Eliminar producto completamente
                cantidad = cantidad_actual #Valor para añadir a la tienda de nuevo
            else:
                carrito.eliminar_producto(producto, cantidad)  # Eliminar parcialmente

            producto.stock += cantidad  # Añadir la cantidad eliminada al stock
            print(f"Producto '{nombre_producto}' eliminado del carrito.")
        else:
            print(f"Producto '{nombre_producto}' no encontrado en el carrito.")

    def crear_carrito(self):
        return Carrito()  # Devuelve una nueva instancia de Carrito

    def mostrar_tienda(self):
        self.tienda.mostrar_tienda()  # Mostrar productos y stock en la tienda

Creamos la tienda y varios productos necesarios

In [ ]:
# Creamos la tienda
tienda1 = Tienda()

# Creamos algunos productos
producto1 = Producto("Manzanas", 1.0, 100)
producto2 = Producto("Plátanos", 1.5, 50)
producto3 = Producto("Naranjas", 2.0, 30)

# Y los añadimos a la tienda
tienda1.añadir_producto(producto1, 100)
tienda1.añadir_producto(producto2, 50)
tienda1.añadir_producto(producto3, 30)

# Mostrar los productos en la tienda
tienda1.mostrar_tienda()



Producto Manzanas añadido o actualizado en la tienda con 100 unidades.
Producto Plátanos añadido o actualizado en la tienda con 50 unidades.
Producto Naranjas añadido o actualizado en la tienda con 30 unidades.
Tienda actual:
Manzanas - Precio: 1.0, Stock: 100
Plátanos - Precio: 1.5, Stock: 50
Naranjas - Precio: 2.0, Stock: 30


Ahora realizamos un par de cambios como si fuesemos el dueño de la tienda



In [ ]:
# Simulamos que el stock de Naranjas llega a cero para comprobar que no desaparece de la tienda
producto3.stock = 0

# Mostrar nuevamente la tienda (ahora con Naranjas en stock 0)
tienda1.mostrar_tienda()

# Intentar eliminar un producto completamente
tienda1.eliminar_producto("Naranjas", eliminar_totalmente=True)  # Eliminar completamente por el dueño
tienda1.mostrar_tienda()  # Verificamos que Naranjas ya no está

# Intentar eliminar otro producto(sin el True del dueño), asi que solo dejándolo con stock 0
tienda1.eliminar_producto("Manzanas")  # Dejar con stock 0
tienda1.mostrar_tienda()  # Verificamos que Manzanas sigue pero con stock 0

tienda1.añadir_producto(producto1, 100)
tienda1.añadir_producto(producto3, 30)
tienda1.mostrar_tienda()  #Volvemos a la situacion inicial

Tienda actual:
Manzanas - Precio: 1.0, Stock: 100
Plátanos - Precio: 1.5, Stock: 50
Naranjas - Precio: 2.0, Stock: 0
Producto Naranjas eliminado completamente de la tienda.
Tienda actual:
Manzanas - Precio: 1.0, Stock: 100
Plátanos - Precio: 1.5, Stock: 50
Producto Manzanas ahora tiene stock 0 pero sigue en la tienda.
Tienda actual:
Manzanas - Precio: 1.0, Stock: 0
Plátanos - Precio: 1.5, Stock: 50
Producto Manzanas añadido o actualizado en la tienda con 100 unidades.
Producto Naranjas añadido o actualizado en la tienda con 30 unidades.
Tienda actual:
Manzanas - Precio: 1.0, Stock: 100
Plátanos - Precio: 1.5, Stock: 50
Naranjas - Precio: 2.0, Stock: 30


In [ ]:
# Creamos la instancia del sistema de compras para la tienda1
sistema1 = SistemaDeCompras(tienda1)

A contiunación nos comportamos como si fuésemos el cliente que interactua con el sistema de compras. Primero creamos el carrito y visualizamos los productos que hay en la tienda

In [ ]:
# Crear el carrito de compras
carrito1 = sistema1.crear_carrito()
# Mostrar tiendas disponibles
print("\n--- Mostrar tiendas disponibles ---")
sistema1.mostrar_tienda()


--- Mostrar tiendas disponibles ---
Tienda actual:
Manzanas - Precio: 1.0, Stock: 100
Plátanos - Precio: 1.5, Stock: 50
Naranjas - Precio: 2.0, Stock: 30


In [ ]:
# Cliente añade productos al carrito
print("\n--- Añadir productos al carrito ---")
sistema1.añadir_al_carrito(carrito1, "Manzanas", 10)
sistema1.añadir_al_carrito(carrito1, "Plátanos", 5)
sistema1.añadir_al_carrito(carrito1, "Naranjas", 15)

# Mostrar información del carrito
print("\n--- Información del carrito después de añadir productos ---")
carrito1.mostrar_informacion()
tienda1.mostrar_tienda()  #Volvemos a la situacion inicial


--- Añadir productos al carrito ---
Producto 'Manzanas' añadido al carrito.
Producto 'Plátanos' añadido al carrito.
Producto 'Naranjas' añadido al carrito.

--- Información del carrito después de añadir productos ---
Carrito de compras:
Manzanas: 10 unidades - Total: 10.0€
Plátanos: 5 unidades - Total: 7.5€
Naranjas: 15 unidades - Total: 30.0€
Coste total: 47.5€
Tienda actual:
Manzanas - Precio: 1.0, Stock: 90
Plátanos - Precio: 1.5, Stock: 45
Naranjas - Precio: 2.0, Stock: 15


In [ ]:
# Cliente elimina algunos productos del carrito
print("\n--- Eliminar productos del carrito (parcialmente) ---")
sistema1.eliminar_del_carrito(carrito1, "Plátanos", 2)  # Eliminar 2 unidades de plátanos
carrito1.mostrar_informacion()

# Eliminar un producto completamente del carrito
print("\n--- Eliminar un producto completamente del carrito ---")
sistema1.eliminar_del_carrito(carrito1, "Manzanas")  # Eliminar todas las manzanas
carrito1.mostrar_informacion()

# Intentar eliminar un producto que no está en el carrito
print("\n--- Intentar eliminar un producto que no está en el carrito ---")
sistema1.eliminar_del_carrito(carrito1, "Peras")  # Producto no existente
carrito1.mostrar_informacion()

# Mostrar información de la tienda después de las compras
tienda1.mostrar_tienda()


--- Eliminar productos del carrito (parcialmente) ---
Producto 'Plátanos' eliminado del carrito.
Carrito de compras:
Manzanas: 10 unidades - Total: 10.0€
Plátanos: 3 unidades - Total: 4.5€
Naranjas: 15 unidades - Total: 30.0€
Coste total: 44.5€

--- Eliminar un producto completamente del carrito ---
Producto 'Manzanas' eliminado del carrito.
Carrito de compras:
Plátanos: 3 unidades - Total: 4.5€
Naranjas: 15 unidades - Total: 30.0€
Coste total: 34.5€

--- Intentar eliminar un producto que no está en el carrito ---
Producto 'Peras' no encontrado en el carrito.
Carrito de compras:
Plátanos: 3 unidades - Total: 4.5€
Naranjas: 15 unidades - Total: 30.0€
Coste total: 34.5€
Tienda actual:
Manzanas - Precio: 1.0, Stock: 100
Plátanos - Precio: 1.5, Stock: 47
Naranjas - Precio: 2.0, Stock: 15
